# Saturation Limits for Direct Imaging

For each NIRCam filter, find the saturation limit for a 2-frame (CDS) ramp.

This notebook cycles over all SW and LW filters to find saturation limits for various detector readout modes:
 * Full Frame
 * Window
  * SUB64
  * SUB160
  * SUB320
  * SUB400
  * SUB640
 * Stripe modes
  * STRIPE64
  * STRIPE128
  * STRIPE256
 
Saturation limits for both NIRCam modules (A and B) are calculated.

Results are returned an AstroPy Table of the results, saved in a dictionary `tbls_dict`.

In [1]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline

from IPython.display import display, Latex, clear_output
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as patches

In [2]:
# pyNRC and associates
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import S
from astropy.table import Table

pynrc.setup_logging('WARNING', verbose=False)

In [3]:
# Filters
filt_wide_sw = ['F070W', 'F090W', 'F115W', 'F150W', 'F150W2', 'F200W']
filt_wide_lw = ['F277W', 'F322W2', 'F356W', 'F444W']

filt_med_sw = ['F140M', 'F162M', 'F182M', 'F210M']
filt_med_lw = ['F250M', 'F300M', 'F335M', 'F360M', 'F410M', 'F430M', 'F460M', 'F480M']

filt_narr_sw = ['F164N', 'F187N', 'F212N']
filt_narr_lw = ['F323N', 'F405N', 'F466N', 'F470N']

filt_all_sw = filt_wide_sw + filt_med_sw + filt_narr_sw
filt_all_lw = filt_wide_lw + filt_med_lw + filt_narr_lw
filt_all = filt_all_sw + filt_all_lw

In [4]:
# NIRCam readout modes
full   = {'wind_mode':'FULL',   'xpix':2048, 'ypix':2048}
# Window subarrays
sub32  = {'wind_mode':'WINDOW', 'xpix':32,   'ypix':32}
sub64  = {'wind_mode':'WINDOW', 'xpix':64,   'ypix':64}
sub160 = {'wind_mode':'WINDOW', 'xpix':160,  'ypix':160}
sub320 = {'wind_mode':'WINDOW', 'xpix':320,  'ypix':320}
sub400 = {'wind_mode':'WINDOW', 'xpix':400,  'ypix':400}
sub640 = {'wind_mode':'WINDOW', 'xpix':640,  'ypix':640}
# Stripe subarrays
stp16  = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':16}
stp32  = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':32}
stp64  = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':64}
stp128 = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':128}
stp256 = {'wind_mode':'STRIPE', 'xpix':2048, 'ypix':256}

subarray_settings = [full,  sub32, sub64, sub160, sub320, sub400, sub640, 
                     stp16, stp32, stp64, stp128, stp256]

# Header names for each readout mode
head = []
head2 = []
for d in subarray_settings:
    if 'FULL' in d['wind_mode']:
        name = 'FULL'
        name2 = name
    elif 'WINDOW' in d['wind_mode']:
        name = 'SUB' + str(d['ypix'])
        name2 = name
    elif 'STRIPE' in d['wind_mode']:
        name = 'STRIPE' + str(d['ypix'])
        name2 = 'STRP' + str(d['ypix'])
    head.append(name)
    head2.append(name2)

In [11]:
nfilt = len(filt_all)
nsub  = len(subarray_settings)
sat_lims = np.zeros([nfilt,nsub])

# We want the K-Band limiting magnitude for a G2V star
sp = nrc_utils.stellar_spectrum('G2V')
bp_lim = S.ObsBandpass('johnson,k')
bp_lim.name = 'K-Band'

tbls_dict = {}

In [13]:
# Module A
mod = 'A'

print('Module {} \n{} saturation magnitudes assuming {} star\n'.format(mod,bp_lim.name, sp.name))
print('{:<6}'.format('Filter') + ' '.join('{:>7}'.format(t) for t in head2))
for i,filt in enumerate(filt_all):
    for j,subset in enumerate(subarray_settings):
        nrc = pynrc.NIRCam(filt, read_mode='RAPID', ngroup=2, module=mod, **subset)
        lim = nrc.sat_limits(sp=sp, bp_lim=bp_lim, units='vegamag')
        sat_lims[i,j] = lim['satmag']
    print('{:<6}'.format(filt) + ' '.join('{: 7.2f}'.format(s) for s in sat_lims[i,:]))

print("")
# Create astropy Table to hold data
tbl = Table(sat_lims, names=head)
for k in tbl.keys():
    tbl[k].format = '6.2f'

# Add filter column and reorder to front
tbl['Filter'] = filt_all
col_order = ['Filter'] + head
tbl = tbl[col_order]

tbls_dict[mod] = tbl

Module A 
K-Band saturation magnitudes assuming G2V star

Filter   FULL   SUB32   SUB64  SUB160  SUB320  SUB400  SUB640  STRP16  STRP32  STRP64 STRP128 STRP256
F070W   14.95    7.81    9.13   10.99   12.45   12.92   13.93    9.75   10.47   11.21   11.95   12.70
F090W   15.66    8.52    9.84   11.70   13.16   13.63   14.64   10.46   11.18   11.92   12.66   13.41
F115W   15.73    8.59    9.90   11.76   13.22   13.70   14.71   10.53   11.25   11.98   12.73   13.48
F150W   15.63    8.49    9.80   11.66   13.12   13.60   14.60   10.42   11.14   11.88   12.62   13.37
F150W2  16.87    9.73   11.04   12.91   14.37   14.84   15.85   11.67   12.39   13.12   13.87   14.62
F200W   15.06    7.92    9.24   11.10   12.56   13.03   14.04    9.86   10.58   11.32   12.06   12.81
F140M   14.87    7.73    9.04   10.90   12.36   12.84   13.84    9.66   10.38   11.12   11.86   12.61
F162M   14.65    7.51    8.82   10.68   12.14   12.62   13.62    9.44   10.16   10.90   11.64   12.39
F182M   14.61    7.47   

In [14]:
# Module B
mod = 'B'

print('Module {} \n{} saturation magnitudes assuming {} star\n'.format(mod,bp_lim.name, sp.name))
print('{:<6}'.format('Filter') + ' '.join('{:>7}'.format(t) for t in head2))
for i,filt in enumerate(filt_all):
    for j,subset in enumerate(subarray_settings):
        nrc = pynrc.NIRCam(filt, read_mode='RAPID', ngroup=2, module=mod, **subset)
        lim = nrc.sat_limits(sp=sp, bp_lim=bp_lim, units='vegamag')
        sat_lims[i,j] = lim['satmag']
    print('{:<6}'.format(filt) + ' '.join('{: 7.2f}'.format(s) for s in sat_lims[i,:]))

print("")
# Create astropy Table to hold data
tbl = Table(sat_lims, names=head)
for k in tbl.keys():
    tbl[k].format = '6.2f'

# Add filter column and reorder to front
tbl['Filter'] = filt_all
col_order = ['Filter'] + head
tbl = tbl[col_order]

tbls_dict[mod] = tbl

Module B 
K-Band saturation magnitudes assuming G2V star

Filter   FULL   SUB32   SUB64  SUB160  SUB320  SUB400  SUB640  STRP16  STRP32  STRP64 STRP128 STRP256
F070W   14.93    7.79    9.10   10.96   12.42   12.90   13.90    9.72   10.44   11.18   11.92   12.67
F090W   15.69    8.55    9.86   11.72   13.18   13.66   14.67   10.49   11.21   11.94   12.69   13.43
F115W   15.71    8.57    9.89   11.75   13.21   13.68   14.69   10.51   11.23   11.97   12.71   13.46
F150W   15.62    8.48    9.79   11.66   13.12   13.59   14.60   10.42   11.14   11.87   12.62   13.37
F150W2  16.87    9.73   11.04   12.90   14.36   14.84   15.85   11.67   12.39   13.12   13.87   14.61
F200W   15.06    7.92    9.23   11.09   12.55   13.03   14.03    9.85   10.57   11.31   12.05   12.80
F140M   14.87    7.73    9.05   10.91   12.37   12.84   13.85    9.67   10.39   11.13   11.87   12.62
F162M   14.65    7.51    8.82   10.68   12.14   12.62   13.62    9.44   10.17   10.90   11.65   12.39
F182M   14.61    7.47   

In [15]:
print(tbls_dict['A'])

Filter  FULL  SUB32  SUB64  SUB160 ... STRIPE32 STRIPE64 STRIPE128 STRIPE256
------ ------ ------ ------ ------ ... -------- -------- --------- ---------
 F070W  14.95   7.81   9.13  10.99 ...    10.47    11.21     11.95     12.70
 F090W  15.66   8.52   9.84  11.70 ...    11.18    11.92     12.66     13.41
 F115W  15.73   8.59   9.90  11.76 ...    11.25    11.98     12.73     13.48
 F150W  15.63   8.49   9.80  11.66 ...    11.14    11.88     12.62     13.37
F150W2  16.87   9.73  11.04  12.91 ...    12.39    13.12     13.87     14.62
 F200W  15.06   7.92   9.24  11.10 ...    10.58    11.32     12.06     12.81
 F140M  14.87   7.73   9.04  10.90 ...    10.38    11.12     11.86     12.61
 F162M  14.65   7.51   8.82  10.68 ...    10.16    10.90     11.64     12.39
 F182M  14.61   7.47   8.78  10.65 ...    10.13    10.86     11.61     12.36
 F210M  13.93   6.79   8.11   9.97 ...     9.45    10.19     10.93     11.68
   ...    ...    ...    ...    ... ...      ...      ...       ...       ...

In [16]:
print(tbls_dict['B'])

Filter  FULL  SUB32  SUB64  SUB160 ... STRIPE32 STRIPE64 STRIPE128 STRIPE256
------ ------ ------ ------ ------ ... -------- -------- --------- ---------
 F070W  14.93   7.79   9.10  10.96 ...    10.44    11.18     11.92     12.67
 F090W  15.69   8.55   9.86  11.72 ...    11.21    11.94     12.69     13.43
 F115W  15.71   8.57   9.89  11.75 ...    11.23    11.97     12.71     13.46
 F150W  15.62   8.48   9.79  11.66 ...    11.14    11.87     12.62     13.37
F150W2  16.87   9.73  11.04  12.90 ...    12.39    13.12     13.87     14.61
 F200W  15.06   7.92   9.23  11.09 ...    10.57    11.31     12.05     12.80
 F140M  14.87   7.73   9.05  10.91 ...    10.39    11.13     11.87     12.62
 F162M  14.65   7.51   8.82  10.68 ...    10.17    10.90     11.65     12.39
 F182M  14.61   7.47   8.78  10.64 ...    10.13    10.86     11.61     12.36
 F210M  13.92   6.78   8.09   9.95 ...     9.43    10.17     10.91     11.66
   ...    ...    ...    ...    ... ...      ...      ...       ...       ...